### Import and parameter

In [3]:
import shutil
import os
import tqdm as notebook_tqdm
import csv
from PIL import Image
from datasets import load_dataset
from ultralytics import YOLO
from pathlib import Path

### Download from hugging face

The images are put into /silver as they are already preprocessed: properly named and resized to 512 px max side.

Three classes are selected: pizza, spaghetti_bolognese, and spaghetti_carbonara.
Pizza can serve as a class that is more distinct. But both spaghetti dishes can be confused for each other.

30 images are downloaded for training, and 10 for validation. 

It is planned to use the Yolo classification model. Due to that, the folder structure and naming are adapted accordingly, and dataset.yaml is created.

In [ ]:
CLASSES = {
    "pizza": 76,
    "spaghetti_bolognese": 90,
    "spaghetti_carbonara": 91,
}
N_SPLIT = {
    "train": 3,
    "validation": 1,
}
OUT_ROOT = Path("../data/silver")
MAKE_CSV = True
SPLIT_MAP = {"train": "train", "validation": "val"}

def map_labels():
    """Map Food-101 label ids <-> names and keep a stable class order."""
    label_names = load_dataset("ethz/food101", split="train").features["label"].names
    id_to_name = {i: n for i, n in enumerate(label_names)}
    target_pairs = [(id_to_name[food101_id], food101_id) for _, food101_id in CLASSES.items()]
    target_names = [n for n, _ in target_pairs]
    target_ids = {i for _, i in target_pairs}
    return id_to_name, target_names, target_ids

def ensure_dirs(root: Path, target_names):
    for split_food101, split_yolo in SPLIT_MAP.items():
        for cls in target_names:
            (root / split_yolo / cls).mkdir(parents=True, exist_ok=True)

def build_split(split_food101: str, OUT_ROOT: Path, target_names, target_ids, id_to_name):
    """
    Download images for the given Food-101 split and save in YOLO classification layout:
    OUT_ROOT/<train|val>/<class_name>/image.jpg
    Optionally writes a labels.csv at split root (filename,label).
    """
    split_yolo = SPLIT_MAP[split_food101]
    split_dir = OUT_ROOT / split_yolo
    split_dir.mkdir(parents=True, exist_ok=True)

    # counters by class name
    saved_counts = {cls: 0 for cls in target_names}
    target_per_class = N_SPLIT[split_food101]

    ds = load_dataset("ethz/food101", split=split_food101)

    rows = []  # optional CSV

    for idx, ex in enumerate(ds):
        lid = ex["label"]
        if lid not in target_ids:
            continue

        cls_name = id_to_name[lid]
        if saved_counts[cls_name] >= target_per_class:
            continue

        fname = f"{idx:06d}_{lid:02d}_{cls_name}.jpg"
        out_path = split_dir / cls_name / fname
        ex["image"].save(out_path)
        saved_counts[cls_name] += 1
        if MAKE_CSV:
            rows.append([f"{cls_name}/{fname}", cls_name])

        # stop early when all classes hit the quota
        if all(saved_counts[c] >= target_per_class for c in saved_counts):
            break

    # optional: CSV summary for this split
    if MAKE_CSV:
        with open(split_dir / "labels.csv", "w", newline="") as f:
            w = csv.writer(f)
            w.writerow(["filename", "label"])
            w.writerows(rows)

    print(f"[{split_food101}] saved counts:", saved_counts)

def write_dataset_yaml(root: Path, target_names):
    """
    Create dataset.yaml for YOLO classification training.
    """
    names_block = "\n".join([f"  - {n}" for n in target_names])
    yaml_text = f"""# YOLO classification dataset generated from Food-101
path: {root.resolve()}
train: train
val: val
names:
{names_block}
"""
    (root / "dataset.yaml").write_text(yaml_text)
    print(f"dataset.yaml written to: {root / 'dataset.yaml'}")
    print("Class order:", target_names)

def prepare_dataset():
    id_to_name, target_names, target_ids = map_labels()
    ensure_dirs(OUT_ROOT, target_names)
    for split in ["train", "validation"]:
        build_split(split, OUT_ROOT, target_names, target_ids, id_to_name)
    write_dataset_yaml(OUT_ROOT, target_names)
    return (OUT_ROOT / "dataset.yaml").resolve()

prepare_dataset()


/usr/local/python/3.11.13/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


[train] saved counts: {'pizza': 30, 'spaghetti_bolognese': 30, 'spaghetti_carbonara': 30}
[validation] saved counts: {'pizza': 10, 'spaghetti_bolognese': 10, 'spaghetti_carbonara': 10}
dataset.yaml written to: ../data/silver/dataset.yaml
Class order: ['pizza', 'spaghetti_bolognese', 'spaghetti_carbonara']


PosixPath('/workspaces/marktguru-home-assignment/data/silver/dataset.yaml')

### Model parameters

In [ ]:
DATASET_YAML = Path("../data/silver/")  # point to your dataset.yaml
BASE_MODEL = "yolov8n-cls.pt"
EPOCHS = 20
IMG_SIZE = 224
BATCH = 16
DEVICE = "cpu"

### First run of the pre-trained model
Pizza is recognized almost perfectly.
Spaghetti bolognese is missing as a class - confused often with carbonara.
Spaghetti carbonara classification is somewhat worse that by pizza - confused with another dish a couple of times.

In [13]:
model = YOLO(BASE_MODEL)
for cls in ["pizza", "spaghetti_bolognese", "spaghetti_carbonara"]:
    model.predict(
        source=Path(DATASET_YAML) / "val" / cls,
        imgsz=IMG_SIZE,
        device=DEVICE,
        save=True,
    )


image 1/10 /workspaces/marktguru-home-assignment/notebooks/../data/silver/val/pizza/002500_76_pizza.jpg: 224x224 pizza 0.95, carbonara 0.02, burrito 0.01, guacamole 0.00, broccoli 0.00, 15.1ms


image 2/10 /workspaces/marktguru-home-assignment/notebooks/../data/silver/val/pizza/002501_76_pizza.jpg: 224x224 pizza 0.99, broccoli 0.00, acorn_squash 0.00, hot_pot 0.00, wok 0.00, 23.4ms
image 3/10 /workspaces/marktguru-home-assignment/notebooks/../data/silver/val/pizza/002502_76_pizza.jpg: 224x224 pizza 0.99, plate 0.00, potpie 0.00, hot_pot 0.00, pineapple 0.00, 15.2ms
image 4/10 /workspaces/marktguru-home-assignment/notebooks/../data/silver/val/pizza/002503_76_pizza.jpg: 224x224 pizza 0.86, meat_loaf 0.02, velvet 0.02, strawberry 0.02, honeycomb 0.01, 13.2ms
image 5/10 /workspaces/marktguru-home-assignment/notebooks/../data/silver/val/pizza/002504_76_pizza.jpg: 224x224 pizza 0.99, pomegranate 0.01, tray 0.00, strawberry 0.00, trifle 0.00, 15.0ms
image 6/10 /workspaces/marktguru-home-assignment/notebooks/../data/silver/val/pizza/002505_76_pizza.jpg: 224x224 pizza 1.00, toilet_seat 0.00, frying_pan 0.00, trifle 0.00, potpie 0.00, 13.5ms
image 7/10 /workspaces/marktguru-home-assignm

### Train model with our data, and predict

In [ ]:
model = YOLO(BASE_MODEL)
results = model.train(
    data=str(DATASET_YAML),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    device=DEVICE,
)
for cls in ["pizza", "spaghetti_bolognese", "spaghetti_carbonara"]:
    model.predict(
        source=Path(DATASET_YAML) / "val" / cls,
        imgsz=IMG_SIZE,
        device=DEVICE,
        save=True,
    )

In [23]:
model = YOLO("runs/classify/train3/weights/best.pt")
model.predict(
    source=Path(DATASET_YAML) / "upload",
    imgsz=224,
    device="cpu",   # or "0" if you have GPU
    save=True,
)



image 1/15 /workspaces/marktguru-home-assignment/notebooks/../data/silver/upload/11973-spaghetti-carbonara-ii-DDMFS-4x3-6edea51e421e4457ac0c3269f3be5157.jpg: 224x224 spaghetti_bolognese 0.98, spaghetti_carbonara 0.02, pizza 0.00, 13.1ms


image 2/15 /workspaces/marktguru-home-assignment/notebooks/../data/silver/upload/Eq_it-na_pizza-margherita_sep2005_sml.jpg: 224x224 pizza 0.93, spaghetti_bolognese 0.06, spaghetti_carbonara 0.01, 17.1ms
image 3/15 /workspaces/marktguru-home-assignment/notebooks/../data/silver/upload/Homemade-Pizza_EXPS_FT23_376_EC_120123_3.jpg: 224x224 spaghetti_bolognese 0.49, pizza 0.47, spaghetti_carbonara 0.04, 16.3ms
image 4/15 /workspaces/marktguru-home-assignment/notebooks/../data/silver/upload/Pizza-3007395.jpg: 224x224 spaghetti_bolognese 0.61, pizza 0.38, spaghetti_carbonara 0.02, 12.4ms
image 5/15 /workspaces/marktguru-home-assignment/notebooks/../data/silver/upload/Salami-pizza-hero.jpg: 224x224 pizza 0.80, spaghetti_bolognese 0.12, spaghetti_carbonara 0.07, 13.9ms
image 6/15 /workspaces/marktguru-home-assignment/notebooks/../data/silver/upload/Spaghetti-Bolognese-Chicken.jpg: 224x224 spaghetti_bolognese 1.00, pizza 0.00, spaghetti_carbonara 0.00, 12.5ms
image 7/15 /workspaces/marktguru-hom

[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: 'pizza', 1: 'spaghetti_bolognese', 2: 'spaghetti_carbonara'}
 obb: None
 orig_img: array([[[210, 212, 213],
         [210, 212, 213],
         [207, 210, 214],
         ...,
         [ 97, 157, 209],
         [ 84, 151, 206],
         [ 60, 134, 188]],
 
        [[211, 211, 211],
         [210, 212, 212],
         [211, 213, 214],
         ...,
         [ 92, 156, 210],
         [ 75, 147, 201],
         [ 42, 118, 171]],
 
        [[214, 212, 211],
         [214, 212, 211],
         [214, 212, 212],
         ...,
         [ 70, 142, 196],
         [ 85, 160, 216],
         [ 63, 140, 196]],
 
        ...,
 
        [[205, 200, 201],
         [205, 200, 201],
         [207, 201, 202],
         ...,
         [194, 194, 194],
         [194, 194, 194],
         [188, 188, 188]],
 
        [[204, 201, 203],
         [205, 202, 204],
         [208, 203, 205],
         ...,
  

### Validate model

In [ ]:
model = YOLO(BASE_MODEL)
results = model.train(
    data=str(DATASET_YAML),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    device=DEVICE,
    verbose=False
)

val_results = model.val(data=str(DATASET_YAML), imgsz=IMG_SIZE, device=DEVICE)
print("Top-1 Accuracy:", val_results.top1)
print("Top-5 Accuracy:", val_results.top5)
print("Summary dict:", val_results.summary())

model.predict(
    source=Path(DATASET_YAML).parent / "silver/val/spaghetti_carbonara",
    imgsz=IMG_SIZE,
    device=DEVICE,
    save=True,
)

### Predict customly downloaded images

Resize 

In [24]:
def resize_max_side(img: Image.Image, max_side=512):
    w, h = img.size
    if max(w, h) <= max_side:
        return img
    scale = max_side / float(max(w, h))
    new = (int(w*scale), int(h*scale))
    return img.resize(new, Image.LANCZOS)
src, dst = Path("../data/upload"), Path("../data/silver/upload")
exts = {".jpg", ".jpeg", ".png", ".bmp", ".webp"}
for f in src.rglob("*"):
    if f.suffix.lower() in exts:
        out = dst / f.relative_to(src)
        out.parent.mkdir(parents=True, exist_ok=True)
        resize_max_side(Image.open(f), 512).save(out, quality=95)
        print(out)

../data/silver/upload/pizza_02.jpg
../data/silver/upload/spaghetti_carbonara_01.jpg
../data/silver/upload/spaghetti_carbonara_02.jpg
../data/silver/upload/spaghetti_bolognese_05.jpg
../data/silver/upload/pizza_05.jpg
../data/silver/upload/spaghetti_carbonara_03.jpg
../data/silver/upload/spaghetti_bolognese_03.jpeg
../data/silver/upload/pizza_01.jpeg
../data/silver/upload/spaghetti_bolognese_04.jpg
../data/silver/upload/spaghetti_carbonara_04.jpg
../data/silver/upload/spaghetti_bolognese_01.jpeg
../data/silver/upload/pizza_03.jpg
../data/silver/upload/pizza_04.jpg
../data/silver/upload/spaghetti_bolognese_02.jpg
../data/silver/upload/spaghetti_carbonara_05.jpg
